In [1]:
%%javascript
Jupyter.utils.load_extensions('tdb_ext/main')

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import tdb
from tdb.examples import viz
import matplotlib.pyplot as plt
import tensorflow as tf
import urllib
import os
import itertools
from IPython.display import Image, display

In [7]:
def viz_activations(ctx, m):
    plt.imshow(np.reshape(m, (-1, PATCH_SIDE)).T, cmap=plt.cm.gray)        
    
def viz_weights(ctx, m):
    plt.title("weights")    
    s = int(np.ceil(np.sqrt(m.shape[0])))
    viz.viz_square(np.reshape(m, (s,s,-1)).T, normalize=False)    

In [3]:
filename = "/home/veon/temp/kcaptcha/generate/*.png"
filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once(filename), shuffle=True)

reader = tf.WholeFileReader()
key, value = reader.read(filename_queue)

img = tf.image.decode_png(value, channels=1)
img = tf.cast(img, tf.float32) /255.0
#split to patches
batch_size = 12

PATCH_SIDE = 14;
n_input = PATCH_SIDE*PATCH_SIDE

ksizes_ = [1, PATCH_SIDE, PATCH_SIDE, 1]
strides_ = [1, 7, 7, 1]
v_strides_ = [1, PATCH_SIDE, PATCH_SIDE, 1]
rates_ = [1, 1, 1, 1]

#input = tf.expand_dims(img, 0)
img.set_shape([60, 100, 1])
input = tf.train.batch([img], batch_size=batch_size
                                    #,num_threads=1
                                    )

train  = tf.extract_image_patches(input, padding='VALID', ksizes=ksizes_, strides=strides_, rates= rates_)
valid  = tf.extract_image_patches(input, padding='VALID', ksizes=ksizes_, strides=v_strides_, rates= rates_)

#min_after_dequeue = 100
#capacity = min_after_dequeue + 3 * batch_size
#split = tf.train.shuffle_batch(
#      [split], batch_size=batch_size, capacity=capacity,
#      min_after_dequeue=min_after_dequeue)

In [4]:
# Parameters
learning_rate = 0.01
training_epochs = 20
display_step = 1
examples_to_show = 10

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features

SEED = 66478  # Set to None for random seed.
# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

#weights = {
encoder_h1 = tf.Variable(tf.truncated_normal([n_input, n_hidden_1],stddev=0.1,seed=SEED))
encoder_h2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],stddev=0.1,seed=SEED))
decoder_h1 = tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_1],stddev=0.1,seed=SEED))
decoder_h2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_input],stddev=0.1,seed=SEED))
#}
#biases = {
encoder_b1 = tf.Variable(tf.constant(0.1, shape=[n_hidden_1]))
encoder_b2 = tf.Variable(tf.constant(0.1, shape=[n_hidden_2]))
decoder_b1 = tf.Variable(tf.constant(0.1, shape=[n_hidden_1]))
decoder_b2 = tf.Variable(tf.constant(0.1, shape=[n_input]))
#}

In [5]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, encoder_h1),
                                   encoder_b1))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, encoder_h2),
                                   encoder_b2))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, decoder_h1),
                                   decoder_b1))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, decoder_h2),
                                   decoder_b2))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

In [6]:
# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.square(y_true - y_pred))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

# Initializing the variables
init = tf.initialize_all_variables()

In [8]:
# debug plots
p0=tdb.plot_op(viz_activations,inputs=[X])
py_pred = tdb.plot_op(viz_activations,inputs=[y_pred])
ploss=tdb.plot_op(viz.watch_loss,inputs=[loss])
g=tf.get_default_graph()
p1=tdb.plot_op(viz_weights,inputs=[g.as_graph_element(encoder_h1)])


In [9]:
def fetch_data(sess, dset):
    splt = sess.run([dset])
    return np.reshape(splt,[-1,n_input])

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    total_batch = int(3604/batch_size)        
    # Training cycle 
    # after 70 cycles loss chart goes flat
    for epoch in range(70):
        # Loop over all batches
        for i in range(total_batch):
            splt = fetch_data(sess, train)
            _ = sess.run([optimizer], feed_dict={X: splt})
                                    
        # Display logs per epoch step
        if epoch % display_step == 0:
            splt = fetch_data(sess, valid)
            status,result = tdb.debug([loss, ploss], feed_dict={X: splt}, breakpoints=None, break_immediately=False, session=sess)
            print('loss: %f' % (result[0]))                                    
            status,result = tdb.debug([py_pred, p0, p1], feed_dict={X: splt[9:15]}, breakpoints=None, break_immediately=False, session=sess)            

    print("You know what? I've Learned something today...")    
        
    coord.request_stop()
    coord.join(threads)

loss: 0.045522
loss: 0.033099
loss: 0.023963
loss: 0.021996
loss: 0.017339
loss: 0.017342
loss: 0.011881
loss: 0.013792
loss: 0.010705
loss: 0.011110
loss: 0.011845
loss: 0.011355
loss: 0.007747
loss: 0.009166
loss: 0.007947
loss: 0.008290
loss: 0.012617
loss: 0.008053
loss: 0.007998
loss: 0.007564
loss: 0.006998
loss: 0.006121
loss: 0.007500
loss: 0.005799
loss: 0.006176
loss: 0.007025
loss: 0.005048
loss: 0.006404
loss: 0.006822
loss: 0.010609
loss: 0.005985
loss: 0.006793
loss: 0.005828
loss: 0.006587
loss: 0.004914
loss: 0.006409
loss: 0.005385
loss: 0.005969
loss: 0.005169
loss: 0.004942
loss: 0.004320
loss: 0.004508
loss: 0.008287
loss: 0.004057
loss: 0.009261
loss: 0.004399
loss: 0.005355
loss: 0.004394
loss: 0.004941
loss: 0.004613
loss: 0.004209
loss: 0.003914
loss: 0.004626
loss: 0.003859
loss: 0.004775
loss: 0.005616
loss: 0.007001
loss: 0.004186
loss: 0.004323
loss: 0.003935
loss: 0.004188
loss: 0.003846
loss: 0.004431
loss: 0.006279
loss: 0.004312
loss: 0.004315
loss: 0.00

KeyboardInterrupt: 